In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [ ]:
def login_stepstone(driver, email, password):
    driver.get("https://www.stepstone.de/de-DE/candidate/login")
    wait = WebDriverWait(driver, 10)

    try:
        # Cookie-Consent akzeptieren
        accept_button = wait.until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, ".privacy-prompt-button.primary-button.ccmgt_accept_button.rebrand")
            )
        )
        accept_button.click()
        print("Cookie-Consent akzeptiert.")
        time.sleep(1)
    except Exception:
        print("Cookie-Consent Button nicht gefunden oder schon akzeptiert.")

    # E-Mail und Passwort eintragen
    email_input = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "gp-x82nov")))
    password_input = driver.find_element(By.CLASS_NAME, "gp-1yn6g5s")

    email_input.clear()
    email_input.send_keys(email)
    password_input.clear()
    password_input.send_keys(password)

    # Login-Button klicken
    login_button = driver.find_element(By.CLASS_NAME, "gp-1ijlosx")
    login_button.click()

    # Warten bis Profil-Link sichtbar (Login erfolgreich)
    print("Login erfolgreich!")

In [ ]:
def parse_salary(clean_text):
    # Vorverarbeitung: z.B. Klammern entfernen
    clean_text = clean_text.replace('\xa0', ' ').strip()
    clean_text = re.sub(r"\(.*?\)", "", clean_text).strip()

    pattern = r"""
        (?:ab\s*)?
        (\d{1,3}(?:\.?\d{3})*(?:,\d{1,2})?)
        [\s\u00a0]*€[\s\u00a0]*
        (?:[-–—]\s*
        (\d{1,3}(?:\.?\d{3})*(?:,\d{1,2})?)\s*€\s*)?
        (monatlich|jährlich)?
    """

    match = re.search(pattern, clean_text, re.IGNORECASE | re.VERBOSE)
    if not match:
        return "keine Angabe"

    try:
        def parse_euro_number(num_str):
            num_str = num_str.replace('.', '').replace(',', '.')
            return float(num_str)

        lower = parse_euro_number(match.group(1))
        upper = parse_euro_number(match.group(2)) if match.group(2) else lower
        avg_salary = (lower + upper) / 2

        zeitraum = match.group(3).lower() if match.group(3) else "jährlich"

        if zeitraum == "monatlich":
            avg_salary *= 14  # Jahresgehalt mit 14 Monatsgehältern

        avg_salary = round(avg_salary)

        #if avg_salary < 15000:  # Plausibilitätscheck
        #   return "keine Angabe"

        return str(avg_salary)
    except:
        return "keine Angabe"

In [ ]:
def slugify(text: str) -> str:
    """Wandelt Text in URL-Slug um: lowercase, Leerzeichen→-, Umlaute→ ae/oe/ue, ß→ss"""
    slug = text.replace(' ', '-')
    return slug

In [ ]:
def get_total_pages(driver):
    """Ermittelt die maximale Seitenzahl aus der Paginierung (unten auf der Seite)."""
    try:
        pagination = driver.find_elements(By.CSS_SELECTOR, "nav.res-wc9x0t ul li button")
        pages = [int(btn.text) for btn in pagination if btn.text.isdigit()]
        return max(pages) if pages else 1
    except:
        return 1

In [ ]:
def scrape_jobs_stepstone(
        driver,
        query: str = "data analyst",
        location: str = "Berlin",
        radius: int = 30,
        max_pages: int = 1
):
    jobs = []
    wait = WebDriverWait(driver, 10)
    job_slug = slugify(query)
    loc_slug = location

    base_url = (
        "https://www.stepstone.de/jobs/{job_slug}/"
        "in-{loc_slug}?radius={radius}"
        "&searchOrigin=Resultlist_top-search&page={page}"
    )

    for page in range(1, max_pages + 1):
        url = base_url.format(job_slug=job_slug, loc_slug=loc_slug, radius=radius, page=page)
        print(f"[Seite {page}] öffne: {url}")
        driver.get(url)

        time.sleep(2)
        cards = driver.find_elements(By.CSS_SELECTOR, "div.res-aa3b6p")
        if not cards:
            print(f"Keine Jobkarten auf Seite {page}. Breche ab.")
            break

        for card in cards:
            try:
                title = card.find_element(By.CSS_SELECTOR, "div.res-ewgtgq").text.strip()
            except:
                title = "Unbekannt"

            try:
                span = card.find_element(
                    By.CSS_SELECTOR,
                    "div.res-5zx6ot span.res-1bl90s9"
                )
                salary = parse_salary(span.text)
            except:
                salary = "keine Angabe"

            # Ort von Stepstone nicht mehr extrahieren – wir setzen ihn manuell
            jobs.append({
                "job_title": title,
                "city": location,    
                "country": "Germany", 
                "annual_salary": salary

            })

        print(f"Seite {page} abgeschlossen. Gesamtanzahl Jobs: {len(jobs)}")

    return jobs


In [ ]:
options = Options()
options.headless = False
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=options)
email = "wi23b056@technikum-wien.at"
password = "Basinga321!"

# 1) Login
login_stepstone(driver, email, password)
time.sleep(5) # Wait for Login
# 2) Scrape
jobs_berlin = scrape_jobs_stepstone(driver, query="Data Scientist", location="Berlin", radius=30, max_pages=5)
jobs_hamburg = scrape_jobs_stepstone(driver, query="Data Scientist", location="Hamburg", radius=30, max_pages=5)
jobs_muenchen = scrape_jobs_stepstone(driver, query="Data Scientist", location="München", radius=30, max_pages=5)
jobs_koeln = scrape_jobs_stepstone(driver, query="Data Scientist", location="Köln", radius=30, max_pages=5)
jobs_frankfurt = scrape_jobs_stepstone(driver, query="Data Scientist", location="Frankfurt", radius=30, max_pages=5)

In [1]:
jobs = jobs_berlin + jobs_hamburg + jobs_muenchen + jobs_koeln + jobs_frankfurt
df = pd.DataFrame(jobs)
df.head(20)

NameError: name 'jobs_berlin' is not defined

In [ ]:
df.to_csv("data_scientist_germany_stepstone.csv", index=False, encoding="utf-8-sig")
print("data_scientist_germany_stepstone.csv gespeichert")

In [ ]:
"""
options = Options()
email = "wi23b056@technikum-wien.at"
password = "Basinga321!"
options.add_argument('--headless')
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=options)
base_url = "https://www.stepstone.de/jobs/data-analyst?page={page}&searchOrigin=membersarea"
login_stepstone(driver, email, password)
time.sleep(3)
url = base_url.format(page=1)
print(f"Scraping URL: {url}")
driver.get(url)
soup = BeautifulSoup(driver.page_source, "html.parser")
print(f"HTML Code: {soup.prettify()}")
"""